In [ ]:
!nvidia-smi

In [ ]:
!python3

In [7]:
!ls

datasets  experiments_gpu_server  validation_notebooks


In [6]:
import os
os.chdir("..")

In [8]:
!pip install torch torchvision
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

#torch.__version__
#!gcc --version
# opencv is pre-installed on colab


# install detectron2:
#!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

  Using cached torch-1.7.0-cp38-cp38-manylinux1_x86_64.whl (776.8 MB)
  Using cached torchvision-0.8.1-cp38-cp38-manylinux1_x86_64.whl (12.8 MB)
  Using cached numpy-1.19.4-cp38-cp38-manylinux2010_x86_64.whl (14.5 MB)
  Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
Processing /home/inexen/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4/future-0.18.2-py3-none-any.whl
  Using cached Pillow-8.0.1-cp38-cp38-manylinux1_x86_64.whl (2.2 MB)
  Using cached Cython-0.29.21-cp38-cp38-manylinux1_x86_64.whl (1.9 MB)
  Using cached PyYAML-5.1.tar.gz (274 kB)
  Created wheel for pyyaml: filename=PyYAML-5.1-cp38-cp38-linux_x86_64.whl size=44073 sha256=64be6f4d71cbe02e92b1343693956a5de43c424943b1ea1386cbf7e65152ddfc
  Stored in directory: /home/inexen/.cache/pip/wheels/52/dd/2b/10ff8b0ac81b93946bb5fb9e6749bae2dac246506c8774e6cf
Successfully built pyyaml
  Cloning https://github.com/cocodataset/cocoap

In [10]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import torch, torchvision

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

ModuleNotFoundError: No module named 'cv2'

In [ ]:
!pip install tifffile

# Predict with model on test data



In [ ]:
from detectron2.data.datasets import register_coco_instances
#register_coco_instances("val", {}, "datasets/polyps2.json", "datasets/polyps2")
register_coco_instances("train", {}, "datasets/kvasir_train_corrected.json", "datasets/Kvasir-SEG")

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer, SimpleTrainer
from detectron2.evaluation import COCOEvaluator, inference_on_dataset, SemSegEvaluator
from detectron2.data import build_detection_test_loader

experiment = 'mask_cascade_rcnn_ResNeSt_50_FPN_syncBN_1x'

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/" + experiment + ".yaml"))
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 10000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon)

cfg.OUTPUT_DIR = 'experiments_gpu_server/' + experiment

cfg.MODEL.WEIGHTS = os.path.join('experiments_gpu_server/' + experiment, "model_0002799.pth")

cfg.DATASETS.TRAIN = ("train",)

trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
# another equivalent way is to use trainer.test

In [ ]:
path = 'datasets/polyps2/'
val_data = os.listdir(path)
_ = sorted(val_data)

In [ ]:
import time
import tifffile
from detectron2.utils.visualizer import ColorMode
predictor = DefaultPredictor(cfg)

time_taken = []

Dict = {0:'polyp'} 

output_folder = '/prediction_masks_2799/'
os.makedirs(cfg.OUTPUT_DIR+ output_folder, exist_ok=True)
for img_path in val_data: 
    if img_path.endswith('.jpg'):   
        print(img_path)
        im = cv2.imread(path + img_path)
        start_time = time.time()
        outputs = predictor(im)
        end_time = time.time() - start_time
        time_taken.append(end_time)
        txt_fullpath = cfg.OUTPUT_DIR + output_folder + os.path.splitext(img_path)[0]+'.txt'  # eg. 0001

        with open(txt_fullpath, 'w+') as file:
            full_text = ''
            #for i, _ in enumerate(outputs['instances'].get('pred_boxes').tensor):
            x_min = str(int(round(outputs['instances'].get('pred_boxes').tensor[0][0].item())))
            y_min = str(int(round(outputs['instances'].get('pred_boxes').tensor[0][1].item())))
            x_max = str(int(round(outputs['instances'].get('pred_boxes').tensor[0][2].item())))
            y_max = str(int(round(outputs['instances'].get('pred_boxes').tensor[0][3].item())))
            score = str(round(outputs['instances'].get('scores')[0].item(),2))
            object_class = int(outputs['instances'].get('pred_classes')[0].item())
            string = Dict[object_class] + " " + score + " " + x_min + " " + y_min + " " + x_max + " " + y_max + '\n'
            full_text = full_text + string

            
            new_shape = list(im.shape)
            new_shape[2] = 1
            new_shape[0], new_shape[2] = new_shape[2], new_shape[0]
            new_shape[1], new_shape[2] = new_shape[2], new_shape[1]
            out = np.zeros(new_shape)
            #for i, _ in enumerate(outputs["instances"].get('pred_masks')):
                
            object_class_seg = int(outputs['instances'].get('pred_classes')[0].item())
            out[object_class_seg,:,:] = outputs["instances"].get('pred_masks')[0].to("cpu").numpy()*255
            out = out.astype(int)
            out = np.uint8(out)
            tifffile.imsave(cfg.OUTPUT_DIR + output_folder +img_path.split('.')[0]+'.tif', out)
            full_text = full_text[:-2]
            file.write(full_text)

In [ ]:
print(time_taken)

In [ ]:
mean_time_taken = np.mean(time_taken)
mean_fps = 1/mean_time_taken
print("Mean time_taken:", mean_time_taken)
print("Mean FPS: ", mean_fps)